In [1]:
# Import the dependencies
import pandas as pd
import os
import gmaps
import gmaps.datasets
import requests
from config import g_key

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [3]:
#load files
weather_py_data_to_load = os.path.join("weather_data","WeatherPy_vacation.csv")
weather_py_df = pd.read_csv(weather_py_data_to_load)
weather_py_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Rikitea,PF,73.74,broken clouds,-23.12,-134.97,Pension Maro'i
1,4,Port-Gentil,GA,73.40,scattered clouds,-0.72,8.78,Fengshui Residence
2,10,Atuona,PF,80.58,clear sky,-9.80,-139.03,Villa Enata
3,13,Ribeira Grande,PT,68.00,scattered clouds,38.52,-28.70,Quinta da Meia Eira
4,22,Bengkulu,ID,75.87,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
...,...,...,...,...,...,...,...,...
197,549,Miyako,JP,69.62,broken clouds,39.64,141.95,Hotel Route Inn Miyako
198,552,Stuttgart,DE,66.99,overcast clouds,48.78,9.18,Hotel Motel One Stuttgart-Mitte
199,554,Poum,NC,70.84,broken clouds,-20.23,164.02,Gîte kajeon
200,562,Zhigansk,RU,70.95,broken clouds,66.77,123.37,Gostinichnyy Kompleks Aylgy


In [4]:
# Using the template add the hotel marks to the map
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name} at {Max Temp}</dd>
"""

# Store the DataFrame row
hotel_info = [info_box_template.format(**row) for index, row in weather_py_df.iterrows()]

# Add a marker layer map of the vacation spots and a pop-up marker for each city
locations = weather_py_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(38,38), zoom_level = 1.8)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
print("Please list 4 cities that are close to each other on the map")

Please list 4 cities that are close to each other on the map


In [6]:
#prompt for city
city1 = input("Enter first city ")
city2 = input("Enter second city ")
city3 = input("Enter third city ")
city4 = input("Enter fourth city ")

Enter first city Broome
Enter second city Jamestown
Enter third city Fairmont
Enter fourth city Hamilton


In [7]:
#loc for separate databases
city1 = weather_py_df.loc[weather_py_df["City"] == city1]
city2 = weather_py_df.loc[weather_py_df["City"] == city2]
city3 = weather_py_df.loc[weather_py_df["City"] == city3]
city4 = weather_py_df.loc[weather_py_df["City"] == city4]

In [8]:
print(city1)
print(city2)
print(city3)
print(city4)

     City_ID    City Country  Max Temp Current Description    Lat    Lng  \
133      388  Broome      US      77.0          few clouds  42.25 -75.83   

                     Hotel Name  
133  Chenango Valley State Park  
    City_ID       City Country  Max Temp Current Description   Lat    Lng  \
11       43  Jamestown      US     78.01           clear sky  42.1 -79.24   

                        Hotel Name  
11  Hampton Inn & Suites Jamestown  
     City_ID      City Country  Max Temp Current Description    Lat    Lng  \
107      305  Fairmont      US      82.0     overcast clouds  39.49 -80.14   

      Hotel Name  
107  Clarion Inn  
    City_ID      City Country  Max Temp Current Description    Lat    Lng  \
43      130  Hamilton      US      82.0       broken clouds  39.18 -84.53   

                   Hotel Name  
43  Six Acres Bed & Breakfast  


In [9]:
#make tuples of all the cities
city1_co = city1["Lat"].values[0], city1["Lng"].values[0]
city2_co = city2["Lat"].values[0], city2["Lng"].values[0]
city3_co = city3["Lat"].values[0], city3["Lng"].values[0]
city4_co = city4["Lat"].values[0], city4["Lng"].values[0]
print(city1_co)
print(city2_co)
print(city3_co)
print(city4_co)

(42.25, -75.83)
(42.1, -79.24)
(39.49, -80.14)
(39.18, -84.53)


In [10]:
#route between the 4 cities. city 1=A, used waypoints for all the stops
fig = gmaps.figure()
city1_co2city4_co = gmaps.directions_layer(city1_co, city4_co, waypoints =[city2_co,city3_co],travel_mode="DRIVING")
fig.add_layer(city1_co2city4_co)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
#merge the city dataframes
city_merge_df = pd.concat([city1, city2, city3, city4])
city_merge_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
133,388,Broome,US,77.00,few clouds,42.25,-75.83,Chenango Valley State Park
11,43,Jamestown,US,78.01,clear sky,42.10,-79.24,Hampton Inn & Suites Jamestown
107,305,Fairmont,US,82.00,overcast clouds,39.49,-80.14,Clarion Inn
43,130,Hamilton,US,82.00,broken clouds,39.18,-84.53,Six Acres Bed & Breakfast


In [12]:
#popup marker for 4 cities
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name} at {Max Temp}</dd>
"""

# Store the DataFrame row
hotel_info = [info_box_template.format(**row) for index, row in city_merge_df.iterrows()]

locations = city_merge_df[["Lat", "Lng"]]
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(height='420px'))